In [ ]:
import os
import openai
import tiktoken

In [ ]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
#TODO
# fix import
# fix queries


# from eventseries.src.main.query.query_series import WikidataEventSeries
# from eventseries.src.main.parsers.ordiniality_extractor import OrdinalExtractor

# # Query event series
# event_series = WikidataEventSeries()
# event_series.event_series_query()

# # Query events and extract ordinality
# extractor = OrdinalExtractor()
# records = extractor.query_extract_ordinal_from_events()
# extractor.dump_events_with_ordinality(records)

In [1]:
#TODO
# improved imports using os

import json

file1 = open(r'C:\Users\resmi\rwth\kglab\CEUR-WS-Event-Series--SS23--Group-2\eventseries\src\main\resources\openai\event_series.json')

event_series = json.load(file1)

file1.close()

file2 = open(r'C:\Users\resmi\rwth\kglab\CEUR-WS-Event-Series--SS23--Group-2\eventseries\src\main\resources\openai\events.json', encoding="utf8")

events = json.load(file2)

file2.close()

In [ ]:
print("Number of events present in wikidata: ", len(events))
print("Number of event series present in wikidata: ", len(event_series))

In [ ]:
# Drop events without title value
events_with_title = []

for event in events:
    if 'title' in event:
        events_with_title.append(event)

print("number of events with a title value: ", len(events_with_title))



# Drop out event series without title
event_series_with_title = []

for binding in event_series:
    if 'title' in binding:
        event_series_with_title.append(binding)

print("number of event series with a title value: ", len(event_series_with_title))

In [ ]:
# subset events with entry "series"
events_with_title_and_series = []

for binding in events:
    if 'title' in binding and 'series' in binding:
        events_with_title_and_series.append(binding)
    
print("number of events with a title value, and linked to an event series: ", len(events_with_title_and_series))

# compare linked events with event series, drop the events if there is no match, making sure that corresponding event series exist
# "series" : type uri, value wikidata link

event_series_dummy = [event['series'] for event in event_series_with_title]

events_with_title_and_series_bijective = [event for event in events_with_title_and_series if event['series'] in event_series_dummy]

print("number of events with a title value, and with respective event series present in event series list: ",len(events_with_title_and_series_bijective))

In [ ]:
# Split data into training and testing datasets

from sklearn.model_selection import train_test_split

train, test = train_test_split(events_with_title_and_series_bijective, test_size=0.1)

In [ ]:
# drop series related data from test dataset

for item in test:
    if "series" in item:
        del item["series"]

    if "seriesLabel" in item:
        del item["seriesLabel"]

    if "ordinal" in item:
        del item["ordinal"]

In [ ]:
# TODO

# check whether volume is a workshop event or a congerence event (might be colocated)
# matching and extraction, 2 sub routines
# incorporate volume numbers into prompts for easier tracking -> low priority


# an example prompt

conversation = [
    {'role': 'system', 'content': 'You are a human'},

    {'role': 'user', 'content': 'In wikidata, there are about 3400 entries which are interesting to me. Lets call these as "events". Additionally, there are different and fewer group of entries, lets call these as "event series". Almost all events are a part of event series. I will provide real examples later on, but for context, we can draw similarities to this example: if each star wars movies are "events", then the star wars itself is the "event series".'},
    {'role': 'user', 'content': 'In wikidata, the property connecting the events to event series are missing, and my task is to deduct from the title of the event which event series does this event belong to. For humans it is an easy task for sure, but noone wants to edit thousands of entries by hand. This is where you step in.'},
    {'role': 'user', 'content': 'I want you to deduct which event series does the following events belong to. To help you out, i will provide 5 titles of some random events, and their corresponding event series to help you out with the pattern recognition. Then i will provide 5 more events for you to find out the event series for these.'},

    {'role': 'user', 'content': "Event 1 is named '8th Workshop on Managing the Evolution and Preservation of the Data Web (MEPDaW)'"},
    {'role': 'assistant', 'content': 'The event series for Event 1 is "Workshop on Managing the Evolution and Preservation of the Data Web"'},
    
    {'role': 'user', 'content': 'Event 2 is named "EON-SWSC 2008 Evaluation of Ontology-based Tools and the Semantic Web Service Challenge"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 2 is "Workshop on Evaluation of Ontology-based Tools"'},
    
    {'role': 'user', 'content': 'Event 3 is named "3rd International Workshop on Knowledge Discovery on the WEB"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 3 is "International Workshop on Knowledge Discovery on the Web"'},
    
    {'role': 'user', 'content': 'Event 4 is named "14th International Conference on Semantic Web Applications and Tools for Health Care and Life Sciences (SWAT4HCLS 2023)"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 4 is "Workshop on Semantic Web Applications and Tools for Life Sciences"'},
    
    {'role': 'user', 'content': 'Event 5 is named "Eighth Italian Conference on Computational Linguistics"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 5 is "Italian Conference on Computational Linguistics"'},


]


In [ ]:


conversation = [
    {'role': 'system', 'content': 'You are a human'},

    {'role': 'user', 'content': 'In wikidata, there are about 3400 entries which are interesting to me. Lets call these as "events". Additionally, there are different and fewer group of entries, lets call these as "event series". Almost all events are a part of event series. I will provide real examples later on, but for context, we can draw similarities to this example: if each star wars movies are "events", then the star wars itself is the "event series".'},
    {'role': 'user', 'content': 'In wikidata, the property connecting the events to event series are missing, and my task is to deduct from the title of the event which event series does this event belong to. For humans it is an easy task for sure, but noone wants to edit thousands of entries by hand. This is where you step in.'},
    {'role': 'user', 'content': 'I want you to deduct which event series does the following events belong to. To help you out, i will provide titles of some random events, and their corresponding event series to help you out with the pattern recognition. Then i will provide more events for you to find out the event series for these.'},
]

# feed training events & corresponding event series into conversation

for count, item in enumerate(train, start=1):
    conversation.append({'role': 'user', 'content': 'Event ' + str(count) + " is named " + item["title"]})
    conversation.append({'role': 'assistant', 'content': 'The event series for Event ' + str(count) + " is " + item["seriesLabel"]})


In [ ]:
# add 5 unknown events to the prompt
events_sample = events_with_title[:5]
deduced_event_series = []
for event in events_sample:
    conversation.append({'role': 'user', 'content': "The event for you to find event series is '" + event["title"]["value"] + "'"})


    # Send the conversation to ChatGPT
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k',
        messages=conversation,
        stop=None,
        temperature=0.7
    )

# Extract the deduced event series from the response
    deduced_event_series.append(response['choices'][0]['message']['content'])

# Add the assistant's response to the conversation history
    conversation.append({'role': 'assistant', 'content': response['choices'][0]['message']['content']})


In [ ]:
for event in events_sample:
    print(event["title"]["value"])

In [ ]:
for ded in deduced_event_series:
    print(ded)

In [ ]:
# another extended prompt


conversation = [
    {'role': 'system', 'content': 'You are a human'},

    # define event
    {'role': 'user', 'content': 'In wikidata, there are about 3400 entries which are interesting to me. Lets call these as "events". These "events" are scientific events like academic conferences or academic workshops.'},
    
    # define proceedings
    {'role': 'user', 'content': 'After an "event" is concluded, the relevant results or papers presented in these "events" are collected into a "proceeding". Each "proceeding" has a unique "volume number". '},
    
    # define event series
    {'role': 'user', 'content': 'Additionally, there are different and fewer group of entries in wikidata, lets call these as "event series". Almost all "events" are a part of "event series", but it is not a necessity. I will provide real examples later on, but for context, we can draw similarities to this example: if each star wars movies are "events", then the star wars itself is the "event series".'},
    
    # define task
    {'role': 'user', 'content': 'In wikidata, the property connecting the events to event series are missing, and my task is to deduct from the title of the event which event series does this event belong to. For humans it is an easy task for sure, but noone wants to edit thousands of entries by hand. This is where you step in.'},
    {'role': 'user', 'content': 'I want you to deduct which event series does the following events belong to. To help you out, i will provide 5 titles of some random events, and their corresponding event series to help you out with the pattern recognition. Then i will provide 5 more events for you to find out the event series for these.'},

    # define response format
    {'role': 'user', 'content': 'For the responses you give to my prompts, I want you to return in a specific format. I provide the outline below.'},
    
    # TODO
    # more specific workflow
    # 

    {'role': 'user', 'content': 'Use the following step-by-step instructions to respond to user inputs. The user will provide you titles for "events" in quotes.'},
    {'role': 'user', 'content': 'Step 1 - If you detect multiple events in a single title, add into response "Colocated: 1". If you detect a single event in a single title, add "Colocated: 0" instead. Most of the "events" are single events, colocation of multiple events occurs less frequently.'},
    {'role': 'user', 'content': 'Step 2 - If the "event" is a workshop, add "Instance of: Academic Workshop". If the event is a conference, add "Instance of: Academic Conference". If it cannot be deducted from the title provided, add "Instance of: None"'},
    {'role': 'user', 'content': 'Step 3 - Now try to extract "event series" from the given title. The extracted string goes into value field for the key "part of the series": "". When you cannot deduct anything, insert "part of the series": "0" into dictionary. Keep in my that the event series can be a long piece of text, up to 50 words in some cases.'},
    {'role': 'user', 'content': 'Step 4 - Return this Python dictionary in your response.'},

    # provide examples
    {'role': 'user', 'content': "Event 1 is named '8th Workshop on Managing the Evolution and Preservation of the Data Web (MEPDaW)'"},
    {'role': 'assistant', 'content': 'The event series for Event 1 is "Workshop on Managing the Evolution and Preservation of the Data Web"'},
    
    {'role': 'user', 'content': 'Event 2 is named "EON-SWSC 2008 Evaluation of Ontology-based Tools and the Semantic Web Service Challenge"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 2 is "Workshop on Evaluation of Ontology-based Tools"'},
    
    {'role': 'user', 'content': 'Event 3 is named "3rd International Workshop on Knowledge Discovery on the WEB"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 3 is "International Workshop on Knowledge Discovery on the Web"'},
    
    {'role': 'user', 'content': 'Event 4 is named "14th International Conference on Semantic Web Applications and Tools for Health Care and Life Sciences (SWAT4HCLS 2023)"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 4 is "Workshop on Semantic Web Applications and Tools for Life Sciences"'},
    
    {'role': 'user', 'content': 'Event 5 is named "Eighth Italian Conference on Computational Linguistics"'},
    {'role': 'assistant', 'content': 'The event series for Matched event 5 is "Italian Conference on Computational Linguistics"'},
]

In [ ]:
# add 5 unknown events to the prompt
events_sample = events_with_title[:5]
deduced_event_series = []
for event in events_sample:
    conversation.append({'role': 'user', 'content': "The event for you to find event series is '" + event["title"]["value"] + "'"})


    # Send the conversation to ChatGPT
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=conversation,
        stop=None,
        temperature=0.7
    )

# Extract the deduced event series from the response
    deduced_event_series.append(response['choices'][0]['message']['content'])

# Add the assistant's response to the conversation history
    conversation.append({'role': 'assistant', 'content': response['choices'][0]['message']['content']})

In [ ]:
for event in events_sample:
    print(event["title"]["value"])

In [ ]:
for ded in deduced_event_series:
    print(ded)

In [ ]:
# token counter, can be used to estimate costs
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

total_token_count = sum(len(encoding.encode(message["content"])) for message in conversation)

print(f"Total token count for conversation: {total_token_count}")

In [3]:
import json

file3 = open(r'C:\Users\resmi\rwth\kglab\CEUR-WS-Event-Series--SS23--Group-2\eventseries\src\main\resources\openai\events2.json', encoding="utf8")

events2 = json.load(file3)

file3.close()

In [5]:
events2 = events2["results"]["bindings"]

In [7]:
count = 0

for item in events2:
    if "instanceOf" in item:
        count += 1

print("events with instance of value: ", count)
print("total no of events: ", len(events2))

events with instance of value:  3556
total no of events:  3556
